# Q13 Before Superbowl

In [1]:
import json
import datetime
import pandas as pd
import numpy as np
import pytz
pst_tz = pytz.timezone('America/Los_Angeles')


import warnings
warnings.filterwarnings("ignore")

In [2]:
def get_X_y(df,window):
    num_features = 5
    
    tweets_by_hour = df.set_index('TimeStamp').groupby(pd.Grouper(freq='60Min'))
    
    total_hours = len(tweets_by_hour)

    X = np.zeros((total_hours, 5))
    for i, (key, val) in enumerate(tweets_by_hour):
            features = [len(val), val.nReTweet.sum(), val.nFollowers.sum(), val.nFollowers.max(), key.hour]
            X[i, :] = features
            
    y = X[:, 0][window:]
    # number of tweets is the output as well as the first feature - but have to shift by #window hours
    X = np.nan_to_num(X)
    X_window = np.zeros((total_hours - window, num_features * window))
    
    for i in range(total_hours - window):
        X_window[i, :] = np.concatenate([X[i+k, :] for k in range(window)])
        
    X_window = np.nan_to_num(X_window)
    return X_window, y

In [3]:
piece_time = datetime.datetime(2015, 2, 1, 8, 0).timestamp()
piece_time_pst = datetime.datetime.fromtimestamp(piece_time, pst_tz)
piece_time_2 = datetime.datetime(2015, 2, 1, 20, 0).timestamp()
piece_time_2_pst = datetime.datetime.fromtimestamp(piece_time_2, pst_tz)

In [4]:
def tweet_to_df(tweet_d):
    df = pd.DataFrame(tweet_d, columns=['ID', 'Title', 'nFollowers', 'nReTweet','Ranking Score','impressions','TimeStamp'])
    df.sort_values(by='TimeStamp',inplace = True)
    return df

In [5]:
def extract_tweets_into_df(file):
    d = []
    for line in open(file):
        temp = []
        t = json.loads(line)
        time = datetime.datetime.fromtimestamp(t['citation_date'], pst_tz)
        if(time < piece_time_pst ):
            temp.append(t['tweet']['id'])
            temp.append(t['title'])
            #temp.append(t['user'])
            temp.append(t['author']['followers'])
            temp.append(t['metrics']['citations']['total'])
            temp.append(t['metrics']['ranking_score'])
            temp.append(t['metrics']['impressions'])
            temp.append(datetime.datetime.fromtimestamp(t['citation_date'], pst_tz))
            d.append(temp)
    return tweet_to_df(d)



In [6]:
nfl_df = extract_tweets_into_df("data/tweets_#nfl.txt")

In [7]:
sb_df = extract_tweets_into_df("data/tweets_#superbowl.txt")

In [8]:
sb49_df = extract_tweets_into_df("data/tweets_#sb49.txt")

In [9]:
pat_df = extract_tweets_into_df("data/tweets_#patriots.txt")

In [10]:
hawks_df = extract_tweets_into_df("data/tweets_#gohawks.txt")

In [11]:
go_pat_df = extract_tweets_into_df("data/tweets_#gopatriots.txt")

In [12]:
frame = [nfl_df,sb_df,sb49_df,pat_df,hawks_df,go_pat_df]
#aggregated df
df = pd.concat(frame)

In [13]:
min_time = df.set_index('TimeStamp').index.min()
max_time = df.set_index('TimeStamp').index.max()
print (min_time)
print (max_time)

total_hours = (max_time - min_time)
total_hours = total_hours.to_timedelta64().astype('timedelta64[h]') + 1
print(total_hours)

2015-01-14 00:00:04-08:00
2015-02-01 07:59:59-08:00
440 hours


In [14]:
X,Y = get_X_y(df, 1)

In [15]:
import  itertools 
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPRegressor
from sklearn  import metrics

parameters={
'hidden_layer_sizes': [t for t in itertools.product((10,20,30,40,50,100),repeat=3)]

}
model = MLPRegressor()


In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

In [18]:
def computeMLP(mlp, X_train, y_train,X_test, y_test):
    mlp.fit(X_train, y_train)
    mlp.score(X_test, y_test)
    predictions = mlp.predict(X_test)

    y_pred = mlp.predict(X_test)
    mse = metrics.mean_squared_error(y_test, y_pred)


    #print("MSE: %.4f" % mse)
    return mse

In [17]:
grid = GridSearchCV(estimator=model,param_grid=parameters,n_jobs=-1,verbose=2,cv=KFold(5, shuffle=True))
grid.fit(X_train, y_train)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits
[CV] hidden_layer_sizes=(10, 10, 10) .................................
[CV] hidden_layer_sizes=(10, 10, 10) .................................
[CV] hidden_layer_sizes=(10, 10, 10) .................................
[CV] hidden_layer_sizes=(10, 10, 10) .................................
[CV] hidden_layer_sizes=(10, 10, 10) .................................
[CV] hidden_layer_sizes=(10, 10, 20) .................................
[CV] hidden_layer_sizes=(10, 10, 20) .................................
[CV] hidden_layer_sizes=(10, 10, 20) .................................
[CV] .................. hidden_layer_sizes=(10, 10, 20), total=   0.2s
[CV] .................. hidden_layer_sizes=(10, 10, 20), total=   0.2s
[CV] hidden_layer_sizes=(10, 10, 20) .................................
[CV] hidden_layer_sizes=(10, 10, 20) .................................
[CV] .................. hidden_layer_sizes=(10, 10, 20), total=   0.2s
[CV] hidden_l

[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    0.6s


[CV] .................. hidden_layer_sizes=(10, 10, 10), total=   0.8s
[CV] .................. hidden_layer_sizes=(10, 20, 50), total=   0.0s
[CV] hidden_layer_sizes=(10, 20, 50) .................................
[CV] hidden_layer_sizes=(10, 20, 50) .................................
[CV] .................. hidden_layer_sizes=(10, 20, 40), total=   0.1s
[CV] hidden_layer_sizes=(10, 20, 40) .................................
[CV] .................. hidden_layer_sizes=(10, 20, 50), total=   0.0s
[CV] hidden_layer_sizes=(10, 20, 100) ................................
[CV] .................. hidden_layer_sizes=(10, 20, 50), total=   0.0s
[CV] hidden_layer_sizes=(10, 20, 100) ................................
[CV] .................. hidden_layer_sizes=(10, 20, 50), total=   0.0s
[CV] .................. hidden_layer_sizes=(10, 20, 40), total=   0.1s
[CV] .................. hidden_layer_sizes=(10, 10, 10), total=   0.8s
[CV] hidden_layer_sizes=(10, 20, 100) ................................
[CV] h

[CV] hidden_layer_sizes=(10, 40, 50) .................................
[CV] hidden_layer_sizes=(10, 40, 20) .................................
[CV] .................. hidden_layer_sizes=(10, 40, 30), total=   0.1s
[CV] .................. hidden_layer_sizes=(10, 40, 30), total=   0.1s
[CV] hidden_layer_sizes=(10, 40, 30) .................................
[CV] hidden_layer_sizes=(10, 40, 40) .................................
[CV] ................. hidden_layer_sizes=(10, 40, 100), total=   0.1s
[CV] hidden_layer_sizes=(10, 40, 100) ................................
[CV] .................. hidden_layer_sizes=(10, 40, 40), total=   0.0s
[CV] hidden_layer_sizes=(10, 40, 40) .................................
[CV] .................. hidden_layer_sizes=(10, 40, 20), total=   0.1s
[CV] .................. hidden_layer_sizes=(10, 40, 50), total=   0.1s
[CV] hidden_layer_sizes=(10, 50, 10) .................................
[CV] hidden_layer_sizes=(10, 50, 10) .................................
[CV] .

[CV] ................. hidden_layer_sizes=(10, 100, 40), total=   0.0s
[CV] ................. hidden_layer_sizes=(10, 100, 20), total=   0.2s
[CV] hidden_layer_sizes=(10, 100, 40) ................................
[CV] hidden_layer_sizes=(10, 100, 20) ................................
[CV] ................ hidden_layer_sizes=(10, 100, 100), total=   0.1s
[CV] hidden_layer_sizes=(10, 100, 100) ...............................
[CV] ................. hidden_layer_sizes=(10, 50, 100), total=   0.2s
[CV] ................. hidden_layer_sizes=(10, 100, 50), total=   0.1s
[CV] ................. hidden_layer_sizes=(10, 100, 20), total=   0.0s
[CV] hidden_layer_sizes=(10, 100, 10) ................................
[CV] hidden_layer_sizes=(20, 10, 10) .................................
[CV] hidden_layer_sizes=(10, 100, 50) ................................
[CV] ................. hidden_layer_sizes=(10, 100, 50), total=   0.1s
[CV] ................ hidden_layer_sizes=(10, 100, 100), total=   0.0s
[CV] h

...

[CV] ................. hidden_layer_sizes=(100, 20, 10), total=   0.0s
[CV] hidden_layer_sizes=(100, 20, 20) ................................
[CV] ................. hidden_layer_sizes=(100, 20, 30), total=   0.1s
[CV] hidden_layer_sizes=(100, 20, 30) ................................
[CV] ................. hidden_layer_sizes=(100, 20, 10), total=   0.1s
[CV] hidden_layer_sizes=(100, 20, 10) ................................
[CV] ................. hidden_layer_sizes=(100, 20, 30), total=   0.0s
[CV] hidden_layer_sizes=(100, 20, 30) ................................
[CV] ................. hidden_layer_sizes=(100, 20, 10), total=   0.0s
[CV] hidden_layer_sizes=(100, 20, 30) ................................
[CV] ................. hidden_layer_sizes=(100, 20, 30), total=   0.0s
[CV] ................. hidden_layer_sizes=(100, 20, 20), total=   0.1s
[CV] ................. hidden_layer_sizes=(100, 20, 20), total=   0.2s
[CV] hidden_layer_sizes=(100, 20, 20) ................................
[CV] h

[CV] hidden_layer_sizes=(100, 40, 20) ................................
[CV] hidden_layer_sizes=(100, 30, 100) ...............................
[CV] ................. hidden_layer_sizes=(100, 40, 10), total=   0.0s
[CV] ................ hidden_layer_sizes=(100, 30, 100), total=   0.0s
[CV] hidden_layer_sizes=(100, 40, 20) ................................
[CV] hidden_layer_sizes=(100, 30, 100) ...............................
[CV] ................. hidden_layer_sizes=(100, 30, 30), total=   0.1s
[CV] ................. hidden_layer_sizes=(100, 30, 40), total=   0.1s
[CV] hidden_layer_sizes=(100, 30, 40) ................................
[CV] ................. hidden_layer_sizes=(100, 40, 20), total=   0.0s
[CV] hidden_layer_sizes=(100, 40, 30) ................................
[CV] ................. hidden_layer_sizes=(100, 40, 20), total=   0.1s
[CV] hidden_layer_sizes=(100, 40, 30) ................................
[CV] hidden_layer_sizes=(100, 40, 20) ................................
[CV] .

[CV] hidden_layer_sizes=(100, 100, 30) ...............................
[CV] ................ hidden_layer_sizes=(100, 50, 100), total=   0.1s
[CV] hidden_layer_sizes=(100, 50, 50) ................................
[CV] hidden_layer_sizes=(100, 50, 100) ...............................
[CV] ................. hidden_layer_sizes=(100, 50, 40), total=   0.1s
[CV] ................ hidden_layer_sizes=(100, 100, 10), total=   0.0s
[CV] hidden_layer_sizes=(100, 100, 10) ...............................
[CV] hidden_layer_sizes=(100, 50, 40) ................................
[CV] ................ hidden_layer_sizes=(100, 50, 100), total=   0.1s
[CV] hidden_layer_sizes=(100, 50, 100) ...............................
[CV] ................. hidden_layer_sizes=(100, 50, 40), total=   0.1s
[CV] hidden_layer_sizes=(100, 100, 30) ...............................
[CV] ................. hidden_layer_sizes=(100, 50, 50), total=   0.1s
[CV] hidden_layer_sizes=(100, 50, 50) ................................
[CV] .

[Parallel(n_jobs=-1)]: Done 1080 out of 1080 | elapsed:   13.9s finished


GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=True),
       error_score='raise',
       estimator=MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'hidden_layer_sizes': [(10, 10, 10), (10, 10, 20), (10, 10, 30), (10, 10, 40), (10, 10, 50), (10, 10, 100), (10, 20, 10), (10, 20, 20), (10, 20, 30), (10, 20, 40), (10, 20, 50), (10, 20, 100), (10, 30, 10), (10, 30, 20), (10, 30, 30), (10, 30, 40), (10, 30, 50), (10, 30, 100), (10, 40, 1...), (100, 100, 10), (100, 100, 20), (100, 100, 30), (100, 100, 40), (100, 100, 50), (100, 100, 100)

In [19]:
print(grid.best_score_) 
print(grid.best_params_)

-204.979565827
{'hidden_layer_sizes': (100, 10, 50)}


In [20]:
mlp = MLPRegressor(hidden_layer_sizes=(100, 10, 50))
mse = computeMLP(mlp,X_train, y_train,X_test, y_test )
print("MSE Value of arch #4 %.4f"%mse)

MSE Value of arch #4 15694161.9519


# Aggregated Data - During Super Bowl (5 mins window)

In [21]:
def extract_tweets_into_df(file):
    d = []
    for line in open(file):
        temp = []
        t = json.loads(line)
        time = datetime.datetime.fromtimestamp(t['citation_date'], pst_tz)
        if(time > piece_time_pst and time < piece_time_2_pst):
            temp.append(t['tweet']['id'])
            temp.append(t['title'])
            #temp.append(t['user'])
            temp.append(t['author']['followers'])
            temp.append(t['metrics']['citations']['total'])
            temp.append(t['metrics']['ranking_score'])
            temp.append(t['metrics']['impressions'])
            temp.append(datetime.datetime.fromtimestamp(t['citation_date'], pst_tz))
            d.append(temp)
    return tweet_to_df(d)



In [22]:
nfl_df = extract_tweets_into_df("data/tweets_#nfl.txt")

In [23]:
sb_df = extract_tweets_into_df("data/tweets_#superbowl.txt")
sb49_df = extract_tweets_into_df("data/tweets_#sb49.txt")
pat_df = extract_tweets_into_df("data/tweets_#patriots.txt")
hawks_df = extract_tweets_into_df("data/tweets_#gohawks.txt")
go_pat_df = extract_tweets_into_df("data/tweets_#gopatriots.txt")


In [24]:
frame = [nfl_df,sb_df,sb49_df,pat_df,hawks_df,go_pat_df]
#aggregated df
df = pd.concat(frame)

min_time = df.set_index('TimeStamp').index.min()
max_time = df.set_index('TimeStamp').index.max()
print (min_time)
print (max_time)

total_hours = (max_time - min_time)
total_hours = total_hours.to_timedelta64().astype('timedelta64[h]') + 1
print(total_hours)

2015-02-01 08:00:01-08:00
2015-02-01 19:59:59-08:00
12 hours


In [25]:
X,Y = get_X_y(df, 5)

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)


In [27]:
grid = GridSearchCV(estimator=model,param_grid=parameters,n_jobs=-1,verbose=2,cv=KFold(5, shuffle=True))
grid.fit(X_train, y_train)




Fitting 5 folds for each of 216 candidates, totalling 1080 fits
[CV] hidden_layer_sizes=(10, 10, 10) .................................
[CV] hidden_layer_sizes=(10, 10, 10) .................................
[CV] hidden_layer_sizes=(10, 10, 10) .................................
[CV] hidden_layer_sizes=(10, 10, 10) .................................
[CV] hidden_layer_sizes=(10, 10, 10) .................................
[CV] hidden_layer_sizes=(10, 10, 20) .................................
[CV] hidden_layer_sizes=(10, 10, 20) .................................
[CV] hidden_layer_sizes=(10, 10, 20) .................................
[CV] .................. hidden_layer_sizes=(10, 10, 10), total=   0.0s
[CV] hidden_layer_sizes=(10, 10, 20) .................................
[CV] .................. hidden_layer_sizes=(10, 10, 20), total=   0.0s
[CV] .................. hidden_layer_sizes=(10, 10, 20), total=   0.1s
[CV] hidden_layer_sizes=(10, 10, 20) .................................
[CV] hidden_l

[CV] hidden_layer_sizes=(10, 30, 30) .................................
[CV] .................. hidden_layer_sizes=(10, 30, 10), total=   0.0s
[CV] hidden_layer_sizes=(10, 30, 10) .................................
[CV] .................. hidden_layer_sizes=(10, 20, 30), total=   0.0s
[CV] .................. hidden_layer_sizes=(10, 20, 50), total=   0.0s
[CV] hidden_layer_sizes=(10, 20, 30) .................................
[CV] .................. hidden_layer_sizes=(10, 30, 20), total=   0.0s
[CV] hidden_layer_sizes=(10, 30, 40) .................................
[CV] hidden_layer_sizes=(10, 30, 20) .................................
[CV] .................. hidden_layer_sizes=(10, 30, 30), total=   0.0s
[CV] hidden_layer_sizes=(10, 30, 30) .................................
[CV] .................. hidden_layer_sizes=(10, 30, 10), total=   0.0s
[CV] hidden_layer_sizes=(10, 30, 10) .................................
[CV] .................. hidden_layer_sizes=(10, 10, 50), total=   0.0s
[CV] h

[CV] hidden_layer_sizes=(10, 50, 30) .................................
[CV] .................. hidden_layer_sizes=(10, 40, 40), total=   0.0s
[CV] hidden_layer_sizes=(10, 40, 40) .................................
[CV] .................. hidden_layer_sizes=(10, 20, 20), total=   0.1s
[CV] hidden_layer_sizes=(10, 50, 50) .................................
[CV] ................. hidden_layer_sizes=(10, 40, 100), total=   0.0s
[CV] .................. hidden_layer_sizes=(10, 40, 30), total=   0.0s
[CV] hidden_layer_sizes=(10, 40, 100) ................................
[CV] hidden_layer_sizes=(10, 40, 40) .................................
[CV] .................. hidden_layer_sizes=(10, 50, 20), total=   0.0s
[CV] .................. hidden_layer_sizes=(10, 50, 40), total=   0.0s
[CV] .................. hidden_layer_sizes=(10, 50, 50), total=   0.0s
[CV] hidden_layer_sizes=(10, 50, 20) .................................
[CV] hidden_layer_sizes=(10, 50, 40) .................................
[CV] .

[Parallel(n_jobs=-1)]: Done  70 tasks      | elapsed:    0.8s


[CV] .................. hidden_layer_sizes=(10, 50, 30), total=   0.0s
[CV] hidden_layer_sizes=(10, 100, 10) ................................
[CV] .................. hidden_layer_sizes=(10, 50, 40), total=   0.0s
[CV] hidden_layer_sizes=(10, 50, 30) .................................
[CV] hidden_layer_sizes=(10, 100, 30) ................................
[CV] ................. hidden_layer_sizes=(10, 100, 20), total=   0.0s
[CV] hidden_layer_sizes=(10, 50, 40) .................................
[CV] hidden_layer_sizes=(10, 100, 20) ................................
[CV] .................. hidden_layer_sizes=(10, 50, 50), total=   0.0s
[CV] hidden_layer_sizes=(10, 50, 100) ................................
[CV] .................. hidden_layer_sizes=(10, 50, 40), total=   0.0s
[CV] ................. hidden_layer_sizes=(10, 100, 30), total=   0.0s
[CV] ................. hidden_layer_sizes=(10, 100, 10), total=   0.0s
[CV] hidden_layer_sizes=(10, 100, 30) ................................
[CV] h

...

[CV] ................ hidden_layer_sizes=(50, 100, 100), total=   0.0s
[CV] ................. hidden_layer_sizes=(100, 10, 40), total=   0.1s
[CV] hidden_layer_sizes=(100, 20, 30) ................................
[CV] ................. hidden_layer_sizes=(100, 10, 50), total=   0.0s
[CV] ................. hidden_layer_sizes=(100, 20, 20), total=   0.0s
[CV] hidden_layer_sizes=(100, 10, 50) ................................
[CV] hidden_layer_sizes=(100, 10, 100) ...............................
[CV] hidden_layer_sizes=(100, 20, 20) ................................
[CV] ................. hidden_layer_sizes=(100, 20, 30), total=   0.0s
[CV] hidden_layer_sizes=(100, 20, 30) ................................
[CV] ................. hidden_layer_sizes=(100, 20, 10), total=   0.0s
[CV] hidden_layer_sizes=(100, 20, 10) ................................
[CV] ................. hidden_layer_sizes=(100, 10, 10), total=   0.0s
[CV] hidden_layer_sizes=(100, 20, 40) ................................
[CV] h

[CV] ................. hidden_layer_sizes=(100, 30, 50), total=   0.0s
[CV] ................. hidden_layer_sizes=(100, 30, 30), total=   0.0s
[CV] hidden_layer_sizes=(100, 30, 100) ...............................
[CV] hidden_layer_sizes=(100, 30, 40) ................................
[CV] ................. hidden_layer_sizes=(100, 40, 10), total=   0.0s
[CV] ................. hidden_layer_sizes=(100, 30, 20), total=   0.0s
[CV] hidden_layer_sizes=(100, 40, 20) ................................
[CV] ................ hidden_layer_sizes=(100, 30, 100), total=   0.0s
[CV] ................. hidden_layer_sizes=(100, 30, 10), total=   0.0s
[CV] ................. hidden_layer_sizes=(100, 40, 30), total=   0.0s
[CV] hidden_layer_sizes=(100, 40, 10) ................................
[CV] hidden_layer_sizes=(100, 30, 100) ...............................
[CV] hidden_layer_sizes=(100, 40, 40) ................................
[CV] ................ hidden_layer_sizes=(100, 30, 100), total=   0.0s
[CV] h

[CV] hidden_layer_sizes=(100, 50, 50) ................................
[CV] hidden_layer_sizes=(100, 100, 20) ...............................
[CV] ................. hidden_layer_sizes=(100, 50, 30), total=   0.0s
[CV] ................. hidden_layer_sizes=(100, 50, 20), total=   0.0s
[CV] hidden_layer_sizes=(100, 50, 20) ................................
[CV] ................ hidden_layer_sizes=(100, 100, 30), total=   0.0s
[CV] hidden_layer_sizes=(100, 50, 40) ................................
[CV] hidden_layer_sizes=(100, 100, 30) ...............................
[CV] ................. hidden_layer_sizes=(100, 50, 40), total=   0.0s
[CV] hidden_layer_sizes=(100, 50, 50) ................................
[CV] ................ hidden_layer_sizes=(100, 50, 100), total=   0.0s
[CV] ................. hidden_layer_sizes=(100, 50, 50), total=   0.0s
[CV] hidden_layer_sizes=(100, 100, 10) ...............................
[CV] ................. hidden_layer_sizes=(100, 50, 20), total=   0.0s
[CV] .

[Parallel(n_jobs=-1)]: Done 1080 out of 1080 | elapsed:    6.1s finished


GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=True),
       error_score='raise',
       estimator=MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'hidden_layer_sizes': [(10, 10, 10), (10, 10, 20), (10, 10, 30), (10, 10, 40), (10, 10, 50), (10, 10, 100), (10, 20, 10), (10, 20, 20), (10, 20, 30), (10, 20, 40), (10, 20, 50), (10, 20, 100), (10, 30, 10), (10, 30, 20), (10, 30, 30), (10, 30, 40), (10, 30, 50), (10, 30, 100), (10, 40, 1...), (100, 100, 10), (100, 100, 20), (100, 100, 30), (100, 100, 40), (100, 100, 50), (100, 100, 100)

In [28]:
print(grid.best_score_)
print(grid.best_params_)


0.0
{'hidden_layer_sizes': (10, 10, 10)}


In [29]:
mlp = MLPRegressor(hidden_layer_sizes=(10, 10, 10))
mse = computeMLP(mlp,X_train, y_train,X_test, y_test )
print("MSE Value of arch #4 %.4f"%mse)

MSE Value of arch #4 406071437752117.0625


# After Super Bowl 

In [30]:
def extract_tweets_into_df(file):
    d = []
    for line in open(file):
        temp = []
        t = json.loads(line)
        time = datetime.datetime.fromtimestamp(t['citation_date'], pst_tz)
        if(time > piece_time_2_pst):
            temp.append(t['tweet']['id'])
            temp.append(t['title'])
            #temp.append(t['user'])
            temp.append(t['author']['followers'])
            temp.append(t['metrics']['citations']['total'])
            temp.append(t['metrics']['ranking_score'])
            temp.append(t['metrics']['impressions'])
            temp.append(datetime.datetime.fromtimestamp(t['citation_date'], pst_tz))
            d.append(temp)
    return tweet_to_df(d)

In [31]:
nfl_df = extract_tweets_into_df("data/tweets_#nfl.txt")
sb_df = extract_tweets_into_df("data/tweets_#superbowl.txt")
sb49_df = extract_tweets_into_df("data/tweets_#sb49.txt")
pat_df = extract_tweets_into_df("data/tweets_#patriots.txt")
hawks_df = extract_tweets_into_df("data/tweets_#gohawks.txt")
go_pat_df = extract_tweets_into_df("data/tweets_#gopatriots.txt")

In [32]:
frame = [nfl_df,sb_df,sb49_df,pat_df,hawks_df,go_pat_df]
#aggregated df
df = pd.concat(frame)

min_time = df.set_index('TimeStamp').index.min()
max_time = df.set_index('TimeStamp').index.max()
print (min_time)
print (max_time)

total_hours = (max_time - min_time)
total_hours = total_hours.to_timedelta64().astype('timedelta64[h]') + 1
print(total_hours)

2015-02-01 20:00:01-08:00
2015-02-07 10:55:36-08:00
135 hours


In [33]:
X,Y = get_X_y(df, 1)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

grid = GridSearchCV(estimator=model,param_grid=parameters,n_jobs=-1,verbose=2,cv=KFold(5, shuffle=True))
grid.fit(X_train, y_train)



Fitting 5 folds for each of 216 candidates, totalling 1080 fits
[CV] hidden_layer_sizes=(10, 10, 10) .................................
[CV] hidden_layer_sizes=(10, 10, 10) .................................
[CV] hidden_layer_sizes=(10, 10, 10) .................................
[CV] hidden_layer_sizes=(10, 10, 10) .................................
[CV] hidden_layer_sizes=(10, 10, 10) .................................
[CV] hidden_layer_sizes=(10, 10, 20) .................................
[CV] hidden_layer_sizes=(10, 10, 20) .................................
[CV] hidden_layer_sizes=(10, 10, 20) .................................
[CV] .................. hidden_layer_sizes=(10, 10, 10), total=   0.1s
[CV] .................. hidden_layer_sizes=(10, 10, 10), total=   0.1s
[CV] hidden_layer_sizes=(10, 10, 20) .................................
[CV] hidden_layer_sizes=(10, 10, 20) .................................
[CV] .................. hidden_layer_sizes=(10, 10, 10), total=   0.1s
[CV] ........

[CV] hidden_layer_sizes=(10, 30, 100) ................................
[CV] .................. hidden_layer_sizes=(10, 30, 30), total=   0.0s
[CV] .................. hidden_layer_sizes=(10, 20, 40), total=   0.0s
[CV] ................. hidden_layer_sizes=(10, 20, 100), total=   0.0s
[CV] hidden_layer_sizes=(10, 30, 30) .................................
[CV] .................. hidden_layer_sizes=(10, 40, 10), total=   0.0s
[CV] hidden_layer_sizes=(10, 20, 40) .................................
[CV] hidden_layer_sizes=(10, 20, 100) ................................
[CV] hidden_layer_sizes=(10, 40, 20) .................................
[CV] .................. hidden_layer_sizes=(10, 30, 40), total=   0.0s
[CV] hidden_layer_sizes=(10, 30, 50) .................................
[CV] .................. hidden_layer_sizes=(10, 30, 10), total=   0.0s
[CV] hidden_layer_sizes=(10, 30, 20) .................................
[CV] ................. hidden_layer_sizes=(10, 30, 100), total=   0.0s
[CV] h

[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.4s


[CV] .................. hidden_layer_sizes=(10, 20, 10), total=   0.0s
[CV] .................. hidden_layer_sizes=(10, 40, 30), total=   0.0s
[CV] hidden_layer_sizes=(10, 40, 30) .................................
[CV] hidden_layer_sizes=(10, 40, 50) .................................
[CV] .................. hidden_layer_sizes=(10, 30, 20), total=   0.1s
[CV] hidden_layer_sizes=(10, 30, 20) .................................
[CV] .................. hidden_layer_sizes=(10, 40, 30), total=   0.0s
[CV] .................. hidden_layer_sizes=(10, 40, 50), total=   0.0s
[CV] hidden_layer_sizes=(10, 40, 30) .................................
[CV] hidden_layer_sizes=(10, 40, 50) .................................
[CV] .................. hidden_layer_sizes=(10, 40, 20), total=   0.1s
[CV] .................. hidden_layer_sizes=(10, 30, 50), total=   0.0s
[CV] hidden_layer_sizes=(10, 40, 30) .................................
[CV] hidden_layer_sizes=(10, 30, 50) .................................
[CV] .

[CV] ................. hidden_layer_sizes=(10, 100, 40), total=   0.1s
[CV] hidden_layer_sizes=(10, 100, 40) ................................
[CV] .................. hidden_layer_sizes=(10, 50, 50), total=   0.0s
[CV] hidden_layer_sizes=(20, 10, 30) .................................
[CV] ................. hidden_layer_sizes=(10, 50, 100), total=   0.0s
[CV] ................. hidden_layer_sizes=(10, 100, 50), total=   0.0s
[CV] hidden_layer_sizes=(10, 100, 100) ...............................
[CV] hidden_layer_sizes=(10, 50, 100) ................................
[CV] ................. hidden_layer_sizes=(10, 100, 40), total=   0.0s
[CV] ................. hidden_layer_sizes=(10, 100, 30), total=   0.0s
[CV] hidden_layer_sizes=(10, 100, 40) ................................
[CV] hidden_layer_sizes=(10, 100, 30) ................................
[CV] .................. hidden_layer_sizes=(20, 10, 30), total=   0.0s
[CV] hidden_layer_sizes=(20, 10, 30) .................................
[CV] .

...

[CV] hidden_layer_sizes=(100, 10, 100) ...............................
[CV] hidden_layer_sizes=(100, 50, 40) ................................
[CV] ................. hidden_layer_sizes=(100, 50, 50), total=   0.0s
[CV] hidden_layer_sizes=(100, 50, 50) ................................
[CV] ................ hidden_layer_sizes=(100, 40, 100), total=   0.0s
[CV] hidden_layer_sizes=(100, 50, 10) ................................
[CV] ................. hidden_layer_sizes=(100, 40, 50), total=   0.0s
[CV] hidden_layer_sizes=(100, 40, 50) ................................
[CV] ................ hidden_layer_sizes=(100, 10, 100), total=   0.0s
[CV] ................. hidden_layer_sizes=(100, 40, 40), total=   0.0s
[CV] hidden_layer_sizes=(100, 50, 100) ...............................
[CV] ................. hidden_layer_sizes=(100, 50, 40), total=   0.0s
[CV] hidden_layer_sizes=(100, 100, 10) ...............................
[CV] hidden_layer_sizes=(100, 50, 40) ................................


[Parallel(n_jobs=-1)]: Done 966 tasks      | elapsed:    5.6s


[CV] ................. hidden_layer_sizes=(100, 50, 50), total=   0.0s
[CV] ................. hidden_layer_sizes=(100, 50, 10), total=   0.0s
[CV] hidden_layer_sizes=(100, 50, 10) ................................
[CV] hidden_layer_sizes=(100, 50, 50) ................................
[CV] ................. hidden_layer_sizes=(100, 40, 50), total=   0.0s
[CV] ................ hidden_layer_sizes=(100, 100, 10), total=   0.0s
[CV] hidden_layer_sizes=(100, 100, 30) ...............................
[CV] hidden_layer_sizes=(100, 100, 20) ...............................
[CV] ................. hidden_layer_sizes=(100, 50, 50), total=   0.0s
[CV] hidden_layer_sizes=(100, 50, 50) ................................
[CV] ................. hidden_layer_sizes=(100, 50, 10), total=   0.0s
[CV] ................. hidden_layer_sizes=(100, 50, 40), total=   0.0s
[CV] ................ hidden_layer_sizes=(100, 50, 100), total=   0.0s
[CV] hidden_layer_sizes=(100, 50, 100) ...............................
[CV] h

[Parallel(n_jobs=-1)]: Done 1080 out of 1080 | elapsed:    6.1s finished


GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=True),
       error_score='raise',
       estimator=MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'hidden_layer_sizes': [(10, 10, 10), (10, 10, 20), (10, 10, 30), (10, 10, 40), (10, 10, 50), (10, 10, 100), (10, 20, 10), (10, 20, 20), (10, 20, 30), (10, 20, 40), (10, 20, 50), (10, 20, 100), (10, 30, 10), (10, 30, 20), (10, 30, 30), (10, 30, 40), (10, 30, 50), (10, 30, 100), (10, 40, 1...), (100, 100, 10), (100, 100, 20), (100, 100, 30), (100, 100, 40), (100, 100, 50), (100, 100, 100)

In [34]:
print(grid.best_score_)
print(grid.best_params_)



-11510.8806367
{'hidden_layer_sizes': (10, 10, 10)}


In [35]:
mlp = MLPRegressor(hidden_layer_sizes=(10, 10, 10))
mse = computeMLP(mlp,X_train, y_train,X_test, y_test )
print("MSE Value of arch #4 %.4f"%mse)

MSE Value of arch #4 3587330319.7161
